In [110]:
import os

In [111]:
cd E:\End_to_End_ML_Projects\Predicting_Ocean_Acidification_with_AI\research

E:\End_to_End_ML_Projects\Predicting_Ocean_Acidification_with_AI\research


In [112]:
os.chdir("../")

In [113]:
%pwd

'E:\\End_to_End_ML_Projects\\Predicting_Ocean_Acidification_with_AI'

In [114]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    #alpha: float
    #l1_ratio: float
    max_depth: int
    min_samples_leaf: int
    min_samples_split: int
    n_estimators: int
    target_column: str

In [115]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [116]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestRegressor
        #params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            n_estimators = params.n_estimators,
            max_depth = params.max_depth,
            max_features = params.max_features,
            criterion= params.criterion,
            random_state= params.random_state,
            #alpha = params.alpha,
            #l1_ratio = params.l1_ratio,
            target_column = schema.name
            
        )

        return model_trainer_config

In [117]:
import pandas as pd
import os
from mlProject import logger
from sklearn.linear_model import ElasticNet
import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [118]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        lr = RandomForestRegressor(n_estimators=100,criterion= 'squared_error',max_depth=None,max_features= 1.0,random_state= 42)
        #lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))



In [119]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-12-11 18:52:24,607: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-11 18:52:24,620: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-11 18:52:24,637: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-11 18:52:24,645: INFO: common: created directory at: artifacts]


[2024-12-11 18:52:24,673: INFO: common: created directory at: artifacts/model_trainer]


e:\End_to_End_ML_Projects\Predicting_Ocean_Acidification_with_AI\.env\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
